In [1]:
import pandas as pd

from google.cloud import storage

In [2]:
data = pd.read_csv('https://controllerdata.lacity.org/api/views/g9h8-fvhu/rows.csv', low_memory=False)

In [3]:
df = data[data['DEPARTMENT_TITLE'] == 'CITY ATTORNEY']

df.head()

,RECORD_NBR,PAY_YEAR,DEPARTMENT_NO,DEPARTMENT_TITLE,JOB_CLASS_PGRADE,JOB_TITLE,EMPLOYMENT_TYPE,JOB_STATUS,MOU,MOU_TITLE,REGULAR_PAY,OVERTIME_PAY,ALL_OTHER_PAY,TOTAL_PAY,CITY_RETIREMENT_CONTRIBUTIONS,BENEFIT_PAY,GENDER,ETHNICITY
23268,303030313933,2017,12,CITY ATTORNEY,0595-0,DEPUTY CITY ATTORNEY II,FULL_TIME,ACTIVE,29,DEPUTY CITY ATTORNEYS,24472.00,0.0,50.00,24522.00,7258.40,1188.44,MALE,CAUCASIAN
23443,3030333236,2017,12,CITY ATTORNEY,0581-0,LEGAL SECRETARY II,FULL_TIME,NOT_ACTIVE,03,CLERICAL,71676.54,0.0,291.00,71967.54,21259.26,14111.50,FEMALE,HISPANIC
23473,3030333837,2017,12,CITY ATTORNEY,0552-G,DEPUTY CITY ATTORNEY III,FULL_TIME,NOT_ACTIVE,29,DEPUTY CITY ATTORNEYS,18729.21,0.0,0.00,18729.21,5555.08,2187.81,FEMALE,CAUCASIAN
23474,3030333837,2017,12,CITY ATTORNEY,0596-0,DEPUTY CITY ATTORNEY III,FULL_TIME,NOT_ACTIVE,29,DEPUTY CITY ATTORNEYS,162520.80,0.0,0.00,162520.80,48203.67,14590.08,FEMALE,CAUCASIAN
23530,3030343638,2017,12,CITY ATTORNEY,0565-0,LEGAL ASSISTANT,FULL_TIME,NOT_ACTIVE,01,ADMINISTRATIVE,41339.20,0.0,783.69,42122.89,12261.21,3662.10,FEMALE,ASIAN


In [4]:
cols = ["RECORD_NBR", "JOB_CLASS_PGRADE", "PAY_YEAR", "GENDER", "ETHNICITY", "JOB_TITLE", "JOB_STATUS", "EMPLOYMENT_TYPE"]

In [5]:
df2 = df.groupby(cols)["REGULAR_PAY"].sum().reset_index()

df2.head()

,RECORD_NBR,JOB_CLASS_PGRADE,PAY_YEAR,GENDER,ETHNICITY,JOB_TITLE,JOB_STATUS,EMPLOYMENT_TYPE,REGULAR_PAY
0,303030313933,0595-0,2017,MALE,CAUCASIAN,DEPUTY CITY ATTORNEY II,ACTIVE,FULL_TIME,24472.0
1,303030313933,0595-0,2018,MALE,CAUCASIAN,DEPUTY CITY ATTORNEY II,ACTIVE,FULL_TIME,129221.6
2,303030313933,0595-0,2019,MALE,CAUCASIAN,DEPUTY CITY ATTORNEY II,ACTIVE,FULL_TIME,134094.4
3,303030313933,0595-0,2020,MALE,CAUCASIAN,DEPUTY CITY ATTORNEY II,ACTIVE,FULL_TIME,147395.2
4,303030313933,0595-0,2021,MALE,CAUCASIAN,DEPUTY CITY ATTORNEY II,ACTIVE,FULL_TIME,56707.2


In [6]:
df2 = df.groupby(cols)["REGULAR_PAY"].sum().reset_index()
df2['JOB_CLASS_PGRADE_NUMERIC'] = df2['JOB_CLASS_PGRADE'].rank(method='dense', ascending=True).astype(int)
df2['JOB_CLASS_PGRADE_RANK'] = df2.groupby('RECORD_NBR')['JOB_CLASS_PGRADE_NUMERIC'].rank('dense').astype(int)
df2['NEW_HIRE'] = df2.groupby('RECORD_NBR')['PAY_YEAR'].rank('dense').astype(int)
df2

,RECORD_NBR,JOB_CLASS_PGRADE,PAY_YEAR,GENDER,ETHNICITY,JOB_TITLE,JOB_STATUS,EMPLOYMENT_TYPE,REGULAR_PAY,JOB_CLASS_PGRADE_NUMERIC,JOB_CLASS_PGRADE_RANK,NEW_HIRE
0,303030313933,0595-0,2017,MALE,CAUCASIAN,DEPUTY CITY ATTORNEY II,ACTIVE,FULL_TIME,24472.00,115,1,1
1,303030313933,0595-0,2018,MALE,CAUCASIAN,DEPUTY CITY ATTORNEY II,ACTIVE,FULL_TIME,129221.60,115,1,2
2,303030313933,0595-0,2019,MALE,CAUCASIAN,DEPUTY CITY ATTORNEY II,ACTIVE,FULL_TIME,134094.40,115,1,3
3,303030313933,0595-0,2020,MALE,CAUCASIAN,DEPUTY CITY ATTORNEY II,ACTIVE,FULL_TIME,147395.20,115,1,4
4,303030313933,0595-0,2021,MALE,CAUCASIAN,DEPUTY CITY ATTORNEY II,ACTIVE,FULL_TIME,56707.20,115,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...
11147,393939303933,0595-0,2017,MALE,TWO OR MORE RACES,DEPUTY CITY ATTORNEY II,ACTIVE,FULL_TIME,25144.00,115,1,1
11148,393939303933,0595-0,2018,MALE,TWO OR MORE RACES,DEPUTY CITY ATTORNEY II,ACTIVE,FULL_TIME,132773.60,115,1,2
11149,393939303933,0595-0,2019,MALE,TWO OR MORE RACES,DEPUTY CITY ATTORNEY II,ACTIVE,FULL_TIME,137779.20,115,1,3
11150,393939303933,0595-0,2020,MALE,TWO OR MORE RACES,DEPUTY CITY ATTORNEY II,ACTIVE,FULL_TIME,151448.24,115,1,4


In [7]:
ETHNICITY_MAPPING = {
  'CAUCASIAN': 'CAUCASIAN',
  'HISPANIC': 'HISPANIC',
  'ASIAN': 'ASIAN',
  'BLACK': 'BLACK',
  'FILIPINO': 'ASIAN',
  'OTHER': 'OTHER',
  'TWO OR MORE RACES': 'OTHER',
  'AMERICAN INDIAN': 'OTHER'
}

In [8]:
df2['ETHNICITY'] = df2['ETHNICITY'].str.strip().replace(ETHNICITY_MAPPING)

In [9]:
JOB_TITLE_MAPPING_01 = {
  'ASSISTANT CITY ATTORNEY': 'ASSISTANT CITY ATTORNEY',
  'CHIEF ASSISTANT CITY ATTORNEY': 'ASSISTANT CITY ATTORNEY',
  'CITY ATTORNEY ACCOUNTING CLERK': 'ACCOUNTING CLERK',
  'CITY ATTORNEY ADMINISTRATIVE COORDINATOR I': 'ADMINISTRATIVE COORDINATOR',
  'CITY ATTORNEY ADMINISTRATIVE COORDINATOR II': 'ADMINISTRATIVE COORDINATOR',
  'CITY ATTORNEY ADMINISTRATIVE COORDINATOR III': 'ADMINISTRATIVE COORDINATOR',
  'CITY ATTORNEY ADMINISTRATIVE COORDINATOR IV': 'ADMINISTRATIVE COORDINATOR',
  'CITY ATTORNEY CHIEF ADMINISTRATIVE ASSISTANT': 'CHIEF ADMINISTRATIVE ASSISTANT',
  'CITY ATTORNEY CHIEF INVESTIGATOR': 'CHIEF INVESTIGATOR',
  'CITY ATTORNEY FINANCIAL MANAGER': 'FINANCIAL MANAGER',
  'CITY ATTORNEY INVESTIGATOR I': 'INVESTIGATOR',
  'CITY ATTORNEY INVESTIGATOR II': 'INVESTIGATOR',
  'CITY ATTORNEY INVESTIGATOR III': 'INVESTIGATOR',
  'DEPUTY CITY ATTORNEY I': 'DEPUTY CITY ATTORNEY ',
  'DEPUTY CITY ATTORNEY II': 'DEPUTY CITY ATTORNEY ',
  'DEPUTY CITY ATTORNEY III': 'DEPUTY CITY ATTORNEY ',
  'DEPUTY CITY ATTORNEY IV': 'DEPUTY CITY ATTORNEY ',
  'EXECUTIVE LEGAL SECRETARY I': 'EXECUTIVE LEGAL SECRETARY',
  'HEARING OFFICER CITY ATTORNEY': 'HEARING OFFICER',
  'LEGAL ASSISTANT': 'LEGAL ASSISTANT',
  'LEGAL ASSISTANT I': 'LEGAL ASSISTANT',
  'LEGAL ASSISTANT II': 'LEGAL ASSISTANT',
  'LEGAL CLERK II': 'LEGAL CLERK',
  'LEGAL SECRETARY I': 'LEGAL SECRETARY',
  'LEGAL SECRETARY II': 'LEGAL SECRETARY',
  'LEGAL SECRETARY III': 'LEGAL SECRETARY',
  'PARALEGAL I': 'PARALEGAL',
  'PARALEGAL II': 'PARALEGAL',
  'PARALEGAL III': 'PARALEGAL',
  'PRINCIPAL CLERK CITY ATTORNEY I': 'PRINCIPAL CLERK CITY ATTORNEY',
  'PRINCIPAL CLERK CITY ATTORNEY II': 'PRINCIPAL CLERK CITY ATTORNEY',
  'SENIOR ASSISTANT CITY ATTORNEY': 'ASSISTANT CITY ATTORNEY',
  'SENIOR LEGAL ASSISTANT': 'LEGAL ASSISTANT',
  'SENIOR LEGAL CLERK I': 'LEGAL CLERK',
  'SENIOR LEGAL CLERK II': 'LEGAL CLERK',
  'SENIOR WITNESS SERVICE COORDINATOR': 'WITNESS SERVICE COORDINATOR',
  'WITNESS SERVICE COORDINATOR': 'WITNESS SERVICE COORDINATOR',
  'LEGAL CLERK I': 'LEGAL CLERK'
}

In [10]:
JOB_TITLE_MAPPING_02 = {
  'ASSISTANT CITY ATTORNEY': 'ASSISTANT CITY ATTORNEY',
  'CHIEF ASSISTANT CITY ATTORNEY': 'ASSISTANT CITY ATTORNEY',
  'CITY ATTORNEY ACCOUNTING CLERK': 'ACCOUNTING CLERK',
  'CITY ATTORNEY ADMINISTRATIVE COORDINATOR I': 'ADMINISTRATIVE COORDINATOR',
  'CITY ATTORNEY ADMINISTRATIVE COORDINATOR II': 'ADMINISTRATIVE COORDINATOR',
  'CITY ATTORNEY ADMINISTRATIVE COORDINATOR III': 'ADMINISTRATIVE COORDINATOR',
  'CITY ATTORNEY ADMINISTRATIVE COORDINATOR IV': 'ADMINISTRATIVE COORDINATOR',
  'CITY ATTORNEY CHIEF ADMINISTRATIVE ASSISTANT': 'CHIEF ADMINISTRATIVE ASSISTANT',
  'CITY ATTORNEY CHIEF INVESTIGATOR': 'CHIEF INVESTIGATOR',
  'CITY ATTORNEY FINANCIAL MANAGER': 'FINANCIAL MANAGER',
  'CITY ATTORNEY INVESTIGATOR I': 'INVESTIGATOR',
  'CITY ATTORNEY INVESTIGATOR II': 'INVESTIGATOR',
  'CITY ATTORNEY INVESTIGATOR III': 'INVESTIGATOR',
  'DEPUTY CITY ATTORNEY I': 'DEPUTY CITY ATTORNEY ',
  'DEPUTY CITY ATTORNEY II': 'DEPUTY CITY ATTORNEY ',
  'DEPUTY CITY ATTORNEY III': 'DEPUTY CITY ATTORNEY ',
  'DEPUTY CITY ATTORNEY IV': 'DEPUTY CITY ATTORNEY ',
  'EXECUTIVE LEGAL SECRETARY I': 'EXECUTIVE LEGAL SECRETARY',
  'HEARING OFFICER CITY ATTORNEY': 'HEARING OFFICER',
  'LEGAL ASSISTANT': 'LEGAL ASSISTANT',
  'LEGAL ASSISTANT I': 'LEGAL ASSISTANT',
  'LEGAL ASSISTANT II': 'LEGAL ASSISTANT',
  'LEGAL CLERK II': 'LEGAL CLERK',
  'LEGAL SECRETARY I': 'LEGAL SECRETARY',
  'LEGAL SECRETARY II': 'LEGAL SECRETARY',
  'LEGAL SECRETARY III': 'LEGAL SECRETARY',
  'PARALEGAL I': 'PARALEGAL',
  'PARALEGAL II': 'PARALEGAL',
  'PARALEGAL III': 'PARALEGAL',
  'PRINCIPAL CLERK CITY ATTORNEY I': 'PRINCIPAL CLERK CITY ATTORNEY',
  'PRINCIPAL CLERK CITY ATTORNEY II': 'PRINCIPAL CLERK CITY ATTORNEY',
  'SENIOR ASSISTANT CITY ATTORNEY': 'ASSISTANT CITY ATTORNEY',
  'SENIOR LEGAL ASSISTANT': 'LEGAL ASSISTANT',
  'SENIOR LEGAL CLERK I': 'LEGAL CLERK',
  'SENIOR LEGAL CLERK II': 'LEGAL CLERK',
  'SENIOR WITNESS SERVICE COORDINATOR': 'WITNESS SERVICE COORDINATOR',
  'WITNESS SERVICE COORDINATOR': 'WITNESS SERVICE COORDINATOR',
  'LEGAL CLERK I': 'LEGAL CLERK'
}

In [11]:
df2['GROUPING_01'] = df2['JOB_TITLE'].str.strip().replace(JOB_TITLE_MAPPING_01)
df2['GROUPING_01'] = df2['GROUPING_01'].str.strip()

df2['GROUPING_02'] = df2['JOB_TITLE'].str.strip().replace(JOB_TITLE_MAPPING_02)
df2['GROUPING_02'] = df2['GROUPING_02'].str.strip()

In [12]:
# drop 1st year
new_hires = df2[(df2['PAY_YEAR'] != 2013) & (df2['NEW_HIRE'] == 1)]

# drop duplicates
new_hires = new_hires.drop_duplicates(subset='RECORD_NBR', keep='first')

In [13]:
new_hires.head()

,RECORD_NBR,JOB_CLASS_PGRADE,PAY_YEAR,GENDER,ETHNICITY,JOB_TITLE,JOB_STATUS,EMPLOYMENT_TYPE,REGULAR_PAY,JOB_CLASS_PGRADE_NUMERIC,JOB_CLASS_PGRADE_RANK,NEW_HIRE,GROUPING_01,GROUPING_02
0,303030313933,0595-0,2017,MALE,CAUCASIAN,DEPUTY CITY ATTORNEY II,ACTIVE,FULL_TIME,24472.00,115,1,1,DEPUTY CITY ATTORNEY,DEPUTY CITY ATTORNEY
9,303032393933,0585-0,2018,FEMALE,HISPANIC,LEGAL CLERK I,ACTIVE,FULL_TIME,12117.60,101,1,1,LEGAL CLERK,LEGAL CLERK
13,3030333233,0581-0,2018,FEMALE,HISPANIC,LEGAL SECRETARY II,NOT_ACTIVE,PER_EVENT,0.00,97,1,1,LEGAL SECRETARY,LEGAL SECRETARY
37,303034303134,0560-0,2020,FEMALE,HISPANIC,CITY ATTORNEY INVESTIGATOR II,ACTIVE,FULL_TIME,90406.08,74,1,1,INVESTIGATOR,INVESTIGATOR
60,3030353233,0573-F,2018,FEMALE,BLACK,DEPUTY CITY ATTORNEY IV,NOT_ACTIVE,PER_EVENT,0.00,90,1,1,DEPUTY CITY ATTORNEY,DEPUTY CITY ATTORNEY
